In [1]:
import pandas as pd
import datetime
import numpy as np
import os

In [2]:
samples_names = {'h1n1': 'run230320_UVZ_BA_23-vsp-1337_',
                 'h3n2': 'run230320_UVZ_BA_23-vsp-2129_',
                 'rsv_a': 'run230320_UVZ_BA_23-vsp-1658_',
                 'rsv_b': 'run230320_UVZ_BA_23-vsp-902_', 
                 'sars_cov_2_139': 'uvzsr-BA_24_00002-A09-BA_24_00000139_',
                 'sars_cov_2_122': 'uvzsr-BA_24_00001-G12-BA_24_00000122_',
                 'sars_cov_2_4964': 'run230710_UVZ_BA_23-vsp-4964_'
                 }

samples_id = {'h1n1': '1337',
                 'h3n2': '2129',
                 'rsv_b': '902',
                 'rsv_a': '1658',
                 'sars_cov_2_122': '122',
                 'sars_cov_2_139': '139',
                 'sars_cov_2_4964': '4964'
                 }


cols_scores = ['match_score', 'mismatch_score', 'open_gap_score', 'extend_gap_score']
set_scores_PairwiseAligner_df = pd.DataFrame([#[1, -1, 0, 0],
                                            #   [1, -1, -1, -1] 
                                              [1, -1, -2, -2]
                                              #, [1, -1, -5, -1] 
                                              #,[2, -1, -5, -1]
                                              ], 
                                              columns=cols_scores)

x = datetime.datetime.now()
d = str(x.year) + '_' + str(x.month) + '_' + str(x.day)
# d = '2024_3_28'
# d = '2024_4_16'
print(d)


2024_4_23


# merge tables with scores for trimming tools

In [8]:
# zmenit trimmer '_trimmomatic' alebo '_cutadapt'
trimmers = ['_trimmomatic', '_cutadapt']
trimmer = trimmers[1]


for _, row in set_scores_PairwiseAligner_df.iterrows():
    
    match_score = row['match_score']
    mismatch_score = row['mismatch_score']
    open_gap_score = row['open_gap_score']
    extend_gap_score = row['extend_gap_score']

    pairwiseAligner_set_scores_name = 'match' + str(match_score) + '_mis' + str(mismatch_score) + '_o' \
                                        + str(open_gap_score) + '_e' + str(extend_gap_score)
    
    scores_df_all_viruses = pd.DataFrame(columns=['virus', 'trimmer', 'quality', 'headcrop', 'minlen', 'score', 'match', 'gap', 'mismatch', 'N_count', 
                      'score_agg', 'sample_id', 'pairwiseAligner_set_score', 'consensus_baseline', 'consensus_baseline_virus'])
    
    for virus in samples_names.keys():

        score_path = './data/result_metrics/' + virus + '/' + virus + '_' + samples_names[virus].split('-')[-1] \
                        + pairwiseAligner_set_scores_name + '_' + d + trimmer + '.tsv'
        
        scores_df = pd.read_csv(score_path, sep='\t')
        scores_df['virus'] = virus
        scores_df['sample_id'] = samples_names[virus].split('-')[-1].split('_')[0]
        scores_df['pairwiseAligner_set_score'] = pairwiseAligner_set_scores_name

        # pridaj oznacenie toho vyhodnotenia, ktore predstavuje MBS (porovnavane samo so sebou)
        scores_df['consensus_baseline'] = np.nan
        scores_df['consensus_baseline_virus'] = np.nan
        
        mbs_parameters = (scores_df['quality'] == 20) & (scores_df['headcrop'] == 0) & (scores_df['minlen'] == 45)
        
        scores_df.loc[mbs_parameters, 'consensus_baseline'] = 'MBS'
        scores_df.loc[mbs_parameters, 'consensus_baseline_virus'] = 'MBS' + '_' + virus
        
        # sort by score
        scores_df = scores_df.sort_values('score')
        
        scores_df_all_viruses = pd.concat([scores_df_all_viruses, scores_df])

    path_to_save = './data/result_metrics/results_all_viruses/results_all_viruses_' + pairwiseAligner_set_scores_name + '_' + d + trimmer + '.tsv'
    scores_df_all_viruses.to_csv(path_to_save, sep='\t', index=False)



In [9]:
scores_df_all_viruses

virus   trimmer quality headcrop minlen    score  match gap  \
34             h1n1  cutadapt      30       10     45  13414.0  13423   0   
33             h1n1  cutadapt      30       10     35  13414.0  13423   0   
31             h1n1  cutadapt      15        5     35  13416.0  13424   0   
30             h1n1  cutadapt      20        5     35  13416.0  13424   0   
29             h1n1  cutadapt      20        5     45  13416.0  13424   0   
..              ...       ...     ...      ...    ...      ...    ...  ..   
12  sars_cov_2_4964  cutadapt      20        0      0  29831.0  29839   0   
13  sars_cov_2_4964  cutadapt      15        0     55  29831.0  29839   0   
14  sars_cov_2_4964  cutadapt      15        0     45  29831.0  29839   0   
8   sars_cov_2_4964  cutadapt      20        0     55  29831.0  29839   0   
17  sars_cov_2_4964  cutadapt      10        0     45  29831.0  29839   0   

   mismatch N_count  score_agg sample_id pairwiseAligner_set_score  \
34        9       9    13405.0      1337      match1_mis-1_o-2_e-2   
33        9       9    13405.0      1337      match1_mis-1_o-2_e-2   
31        8       8    13408.0      1337      match1_mis-1_o-2_e-2   
30        8       8    13408.0      1337      match1_mis-1_o-2_e-2   
29        8       8    13408.0      1337      match1_mis-1_o-2_e-2   
..      ...     ...        ...       ...                       ...   
12        8       8    29823.0      4964      match1_mis-1_o-2_e-2   
13        8       8    29823.0      4964      match1_mis-1_o-2_e-2   
14        8       8    29823.0      4964      match1_mis-1_o-2_e-2   
8         8       8    29823.0      4964      match1_mis-1_o-2_e-2   
17        8       8    29823.0      4964      match1_mis-1_o-2_e-2   

   consensus_baseline consensus_baseline_virus  
34                NaN                      NaN  
33                NaN                      NaN  
31                NaN                      NaN  
30                NaN                      NaN  
29                NaN                      NaN  
..                ...                      ...  
12                NaN                      NaN  
13                NaN                      NaN  
14                NaN                      NaN  
8                 NaN                      NaN  
17                NaN                      NaN  

[236 rows x 15 columns]

In [16]:
pairwiseAligner_set_scores_name

'match2_mis-1_o-5_e-1'

In [5]:
scores_df_all_viruses

virus   trimmer quality headcrop minlen    score  match gap mismatch  \
0    h1n1  cutadapt      30       10     45  26861.0  13431   0        1   
1    h1n1  cutadapt      30       10     35  26861.0  13431   0        1   
2    h1n1  cutadapt      30        0      0  26861.0  13431   0        1   
3    h1n1  cutadapt      20        5     45  26858.0  13430   0        2   
4    h1n1  cutadapt      20        5     55  26858.0  13430   0        2   
..    ...       ...     ...      ...    ...      ...    ...  ..      ...   
28  rsv_a  cutadapt      15        0      0  30442.0  15221   0        0   
29  rsv_a  cutadapt      10        0     65  30442.0  15221   0        0   
30  rsv_a  cutadapt      10        0     55  30442.0  15221   0        0   
31  rsv_a  cutadapt      10        0     45  30442.0  15221   0        0   
32  rsv_a  cutadapt      30        0      0  30442.0  15221   0        0   

   N_count  score_agg sample_id pairwiseAligner_set_score consensus_baseline  \
0        9    26860.0      1337      match2_mis-1_o-5_e-1                NaN   
1        9    26860.0      1337      match2_mis-1_o-5_e-1                NaN   
2        7    26860.0      1337      match2_mis-1_o-5_e-1                NaN   
3        8    26856.0      1337      match2_mis-1_o-5_e-1                NaN   
4        8    26856.0      1337      match2_mis-1_o-5_e-1                NaN   
..     ...        ...       ...                       ...                ...   
28       0    30442.0      1658      match2_mis-1_o-5_e-1                NaN   
29       0    30442.0      1658      match2_mis-1_o-5_e-1                NaN   
30       0    30442.0      1658      match2_mis-1_o-5_e-1                NaN   
31       0    30442.0      1658      match2_mis-1_o-5_e-1                NaN   
32       0    30442.0      1658      match2_mis-1_o-5_e-1                NaN   

   consensus_baseline_virus  
0                       NaN  
1                       NaN  
2                       NaN  
3                       NaN  
4                       NaN  
..                      ...  
28                      NaN  
29                      NaN  
30                      NaN  
31                      NaN  
32                      NaN  

[233 rows x 15 columns]

# merge tables with scores for variant callers

In [3]:

for _, row in set_scores_PairwiseAligner_df.iterrows():
    
    match_score = row['match_score']
    mismatch_score = row['mismatch_score']
    open_gap_score = row['open_gap_score']
    extend_gap_score = row['extend_gap_score']

    pairwiseAligner_set_scores_name = 'match' + str(match_score) + '_mis' + str(mismatch_score) + '_o' \
                                        + str(open_gap_score) + '_e' + str(extend_gap_score)
    
    scores_df_all_viruses = pd.DataFrame(columns=['virus', 'variant_caller', 'param', 'param_val', 'score', 'match', 'gap', 'mismatch', 'N_count', 
                      'score_agg', 'sample_id', 'pairwiseAligner_set_score', 'consensus_baseline', 'consensus_baseline_virus'])
    
    for virus in samples_names.keys():

        score_path = './data/result_metrics/' + virus + '/' + virus + '_' + samples_names[virus].split('-')[-1] \
                        + pairwiseAligner_set_scores_name + '_' + d + '_variant_callers.tsv'
        if os.path.exists(score_path):

            scores_df = pd.read_csv(score_path, sep='\t')
            scores_df['virus'] = virus
            scores_df['sample_id'] = samples_id[virus]
            scores_df['pairwiseAligner_set_score'] = pairwiseAligner_set_scores_name

            # pridaj oznacenie toho vyhodnotenia, ktore predstavuje MBS (porovnavane samo so sebou)
            scores_df['consensus_baseline'] = np.nan
            scores_df['consensus_baseline_virus'] = np.nan
            
            mbs_parameters = (scores_df['param'] == 'baseline_params')
            
            scores_df.loc[mbs_parameters, 'consensus_baseline'] = 'MBS'
            scores_df.loc[mbs_parameters, 'consensus_baseline_virus'] = 'MBS' + '_' + virus
            
            # sort by score
            scores_df = scores_df.sort_values(by=['virus', 'variant_caller', 'score'], ascending=[True, True, False])
            
            scores_df_all_viruses = pd.concat([scores_df_all_viruses, scores_df])

    path_to_save = './data/result_metrics/results_all_viruses/results_all_viruses_' + pairwiseAligner_set_scores_name + '_' + d + '_variant_callers.tsv'
    scores_df_all_viruses.to_csv(path_to_save, sep='\t', index=False)



In [6]:
scores_df_all_viruses

virus variant_caller            param param_val    score  match  \
0              h1n1       bcftools        adjust_mq         0  13422.0  13427   
1              h1n1       bcftools            minbq         0  13420.0  13426   
2              h1n1       bcftools            minbq        10  13420.0  13426   
3              h1n1       bcftools            minbq        20  13420.0  13426   
4              h1n1       bcftools            minmq        15  13420.0  13426   
..              ...            ...              ...       ...      ...    ...   
19  sars_cov_2_4964           ivar            minbq        20  29843.0  29845   
20  sars_cov_2_4964           ivar            minmq         0  29843.0  29845   
21  sars_cov_2_4964           ivar            minmq        30  29843.0  29845   
22  sars_cov_2_4964           ivar         redo_baq         1  29843.0  29845   
23  sars_cov_2_4964           ivar  baseline_params         -  29843.0  29845   

   gap mismatch N_count  score_agg sample_id pairwiseAligner_set_score  \
0    0        5       5    13417.0      1337      match1_mis-1_o-2_e-2   
1    0        6       5    13414.0      1337      match1_mis-1_o-2_e-2   
2    0        6       5    13414.0      1337      match1_mis-1_o-2_e-2   
3    0        6       5    13414.0      1337      match1_mis-1_o-2_e-2   
4    0        6       5    13414.0      1337      match1_mis-1_o-2_e-2   
..  ..      ...     ...        ...       ...                       ...   
19   0        2       2    29841.0      4964      match1_mis-1_o-2_e-2   
20   0        2       2    29841.0      4964      match1_mis-1_o-2_e-2   
21   0        2       2    29841.0      4964      match1_mis-1_o-2_e-2   
22   0        2       2    29841.0      4964      match1_mis-1_o-2_e-2   
23   0        2       2    29841.0      4964      match1_mis-1_o-2_e-2   

   consensus_baseline consensus_baseline_virus  
0                 NaN                      NaN  
1                 NaN                      NaN  
2                 NaN                      NaN  
3                 NaN                      NaN  
4                 NaN                      NaN  
..                ...                      ...  
19                NaN                      NaN  
20                NaN                      NaN  
21                NaN                      NaN  
22                NaN                      NaN  
23                MBS      MBS_sars_cov_2_4964  

[175 rows x 14 columns]

# merge tables with scores for trimm_VC comparison

In [6]:

for _, row in set_scores_PairwiseAligner_df.iterrows():
    
    match_score = row['match_score']
    mismatch_score = row['mismatch_score']
    open_gap_score = row['open_gap_score']
    extend_gap_score = row['extend_gap_score']

    pairwiseAligner_set_scores_name = 'match' + str(match_score) + '_mis' + str(mismatch_score) + '_o' \
                                        + str(open_gap_score) + '_e' + str(extend_gap_score)
    
    scores_df_all_viruses = pd.DataFrame(columns=['virus', 'trimmer', 'variant_caller', 'param', 'score', 'match', 'gap', 'mismatch', 'N_count', 
                      'score_agg', 'sample_id', 'pairwiseAligner_set_score', 'consensus_baseline', 'consensus_baseline_virus'])
    
    for virus in samples_names.keys():

        score_path = './data/result_metrics/' + virus + '/' + virus + '_' + samples_names[virus].split('-')[-1] \
                        + pairwiseAligner_set_scores_name + '_' + d + '_trimm_vc.tsv'
        
        if os.path.exists(score_path):

            scores_df = pd.read_csv(score_path, sep='\t')
            scores_df['virus'] = virus
            scores_df['sample_id'] = samples_id[virus]
            scores_df['pairwiseAligner_set_score'] = pairwiseAligner_set_scores_name

            scores_df['consensus_baseline'] = np.nan
            scores_df['consensus_baseline_virus'] = np.nan
            
            
            # sort by score
            scores_df = scores_df.sort_values(by=['virus', 'score'], ascending=[True, False])
            
            scores_df_all_viruses = pd.concat([scores_df_all_viruses, scores_df])
        
        else:
            print('Path: ', score_path, ' does not exists')

    path_to_save = './data/result_metrics/results_all_viruses/results_all_viruses_' + pairwiseAligner_set_scores_name + '_' + d + '_trimm_vc.tsv'
    scores_df_all_viruses.to_csv(path_to_save, sep='\t', index=False)



In [7]:
scores_df_all_viruses

virus      trimmer variant_caller param    score  match gap  \
2             h1n1  trimmomatic      freebayes   NaN  13424.0  13428   0   
3             h1n1     cutadapt      freebayes   NaN  13424.0  13428   0   
4             h1n1  trimmomatic           ivar   NaN  13424.0  13428   0   
5             h1n1     cutadapt           ivar   NaN  13424.0  13428   0   
0             h1n1  trimmomatic       bcftools   NaN  13422.0  13427   0   
1             h1n1     cutadapt       bcftools   NaN  13422.0  13427   0   
2             h3n2  trimmomatic      freebayes   NaN  13179.0  13179   0   
3             h3n2     cutadapt      freebayes   NaN  13179.0  13179   0   
0             h3n2  trimmomatic       bcftools   NaN  13177.0  13178   0   
1             h3n2     cutadapt       bcftools   NaN  13177.0  13178   0   
4             h3n2  trimmomatic           ivar   NaN  13177.0  13178   0   
5             h3n2     cutadapt           ivar   NaN  13177.0  13178   0   
4            rsv_a  trimmomatic           ivar   NaN  15221.0  15221   0   
5            rsv_a     cutadapt           ivar   NaN  15221.0  15221   0   
0            rsv_a  trimmomatic       bcftools   NaN  14793.0  15011   4   
1            rsv_a     cutadapt       bcftools   NaN  14793.0  15011   4   
2            rsv_a  trimmomatic      freebayes   NaN  14793.0  15011   4   
3            rsv_a     cutadapt      freebayes   NaN  14793.0  15011   4   
4            rsv_b  trimmomatic           ivar   NaN  15154.0  15189   0   
5            rsv_b     cutadapt           ivar   NaN  15154.0  15189   0   
0            rsv_b  trimmomatic       bcftools   NaN  14960.0  15096   6   
1            rsv_b     cutadapt       bcftools   NaN  14960.0  15096   6   
2            rsv_b  trimmomatic      freebayes   NaN  14960.0  15096   6   
3            rsv_b     cutadapt      freebayes   NaN  14960.0  15096   6   
4   sars_cov_2_139  trimmomatic           ivar   NaN  29807.0  29821   0   
5   sars_cov_2_139     cutadapt           ivar   NaN  29807.0  29821   0   
0   sars_cov_2_139  trimmomatic       bcftools   NaN  29785.0  29813   3   
1   sars_cov_2_139     cutadapt       bcftools   NaN  29785.0  29813   3   
2   sars_cov_2_139  trimmomatic      freebayes   NaN  29755.0  29821  26   
3   sars_cov_2_139     cutadapt      freebayes   NaN  29755.0  29821  26   
4   sars_cov_2_122  trimmomatic           ivar   NaN  29831.0  29833   0   
5   sars_cov_2_122     cutadapt           ivar   NaN  29831.0  29833   0   
0   sars_cov_2_122     cutadapt       bcftools   NaN  29819.0  29830   3   
1   sars_cov_2_122  trimmomatic       bcftools   NaN  29817.0  29829   3   
2   sars_cov_2_122  trimmomatic      freebayes   NaN  29779.0  29833  26   
3   sars_cov_2_122     cutadapt      freebayes   NaN  29779.0  29833  26   
2  sars_cov_2_4964  trimmomatic      freebayes   NaN  29833.0  29840   0   
3  sars_cov_2_4964     cutadapt      freebayes   NaN  29833.0  29840   0   
4  sars_cov_2_4964  trimmomatic           ivar   NaN  29833.0  29840   0   
5  sars_cov_2_4964     cutadapt           ivar   NaN  29831.0  29839   0   
0  sars_cov_2_4964  trimmomatic       bcftools   NaN  29819.0  29833   0   
1  sars_cov_2_4964     cutadapt       bcftools   NaN  29819.0  29833   0   

  mismatch N_count  score_agg sample_id pairwiseAligner_set_score  \
2        4       4    13420.0      1337      match1_mis-1_o-2_e-2   
3        4       4    13420.0      1337      match1_mis-1_o-2_e-2   
4        4       4    13420.0      1337      match1_mis-1_o-2_e-2   
5        4       4    13420.0      1337      match1_mis-1_o-2_e-2   
0        5       4    13417.0      1337      match1_mis-1_o-2_e-2   
1        5       4    13417.0      1337      match1_mis-1_o-2_e-2   
2        0       1    13179.0      2129      match1_mis-1_o-2_e-2   
3        0       1    13179.0      2129      match1_mis-1_o-2_e-2   
0        1       1    13176.0      2129      match1_mis-1_o-2_e-2   
1        1       1    13176.0      2129      match1_mi

In [5]:
for virus in samples_names.keys():
    max_score_virus = scores_df_all_viruses[scores_df_all_viruses.virus == virus].score.max()
    print(virus)
    print(scores_df_all_viruses[(scores_df_all_viruses.virus == virus) & (scores_df_all_viruses.score == max_score_virus)].variant_caller)

h1n1
2    freebayes
3    freebayes
4         ivar
5         ivar
Name: variant_caller, dtype: object
h3n2
2    freebayes
3    freebayes
Name: variant_caller, dtype: object
rsv_a
4    ivar
5    ivar
Name: variant_caller, dtype: object
rsv_b
4    ivar
5    ivar
Name: variant_caller, dtype: object
sars_cov_2_139
4    ivar
5    ivar
Name: variant_caller, dtype: object
sars_cov_2_122
4    ivar
5    ivar
Name: variant_caller, dtype: object
sars_cov_2_4964
2    freebayes
3    freebayes
4         ivar
Name: variant_caller, dtype: object


In [16]:
for virus in samples_names.keys():
    max_score_virus = scores_df_all_viruses[scores_df_all_viruses.virus == virus].score.max()
    print(virus)
    print(scores_df_all_viruses[(scores_df_all_viruses.virus == virus) & (scores_df_all_viruses.score == max_score_virus)].variant_caller, scores_df_all_viruses[(scores_df_all_viruses.virus == virus) & (scores_df_all_viruses.score == max_score_virus)].trimmer)

h1n1
2    freebayes
3    freebayes
4         ivar
5         ivar
Name: variant_caller, dtype: object 2    trimmomatic
3       cutadapt
4    trimmomatic
5       cutadapt
Name: trimmer, dtype: object
h3n2
2    freebayes
3    freebayes
Name: variant_caller, dtype: object 2    trimmomatic
3       cutadapt
Name: trimmer, dtype: object
rsv_a
4    ivar
5    ivar
Name: variant_caller, dtype: object 4    trimmomatic
5       cutadapt
Name: trimmer, dtype: object
rsv_b
4    ivar
5    ivar
Name: variant_caller, dtype: object 4    trimmomatic
5       cutadapt
Name: trimmer, dtype: object
sars_cov_2_139
4    ivar
5    ivar
Name: variant_caller, dtype: object 4    trimmomatic
5       cutadapt
Name: trimmer, dtype: object
sars_cov_2_122
4    ivar
5    ivar
Name: variant_caller, dtype: object 4    trimmomatic
5       cutadapt
Name: trimmer, dtype: object
sars_cov_2_4964
2    freebayes
3    freebayes
4         ivar
Name: variant_caller, dtype: object 2    trimmomatic
3       cutadapt
4    trimmomatic
Na

In [12]:
scores_df_all_viruses

virus      trimmer variant_caller param    score  match gap  \
2             h1n1  trimmomatic      freebayes   NaN  13424.0  13428   0   
3             h1n1     cutadapt      freebayes   NaN  13424.0  13428   0   
4             h1n1  trimmomatic           ivar   NaN  13424.0  13428   0   
5             h1n1     cutadapt           ivar   NaN  13424.0  13428   0   
0             h1n1  trimmomatic       bcftools   NaN  13422.0  13427   0   
1             h1n1     cutadapt       bcftools   NaN  13422.0  13427   0   
2             h3n2  trimmomatic      freebayes   NaN  13179.0  13179   0   
3             h3n2     cutadapt      freebayes   NaN  13179.0  13179   0   
0             h3n2  trimmomatic       bcftools   NaN  13177.0  13178   0   
1             h3n2     cutadapt       bcftools   NaN  13177.0  13178   0   
4             h3n2  trimmomatic           ivar   NaN  13177.0  13178   0   
5             h3n2     cutadapt           ivar   NaN  13177.0  13178   0   
4            rsv_a  trimmomatic           ivar   NaN  15221.0  15221   0   
5            rsv_a     cutadapt           ivar   NaN  15221.0  15221   0   
0            rsv_a  trimmomatic       bcftools   NaN  14801.0  15015  12   
1            rsv_a     cutadapt       bcftools   NaN  14801.0  15015  12   
2            rsv_a  trimmomatic      freebayes   NaN  14801.0  15015  12   
3            rsv_a     cutadapt      freebayes   NaN  14801.0  15015  12   
4            rsv_b  trimmomatic           ivar   NaN  15154.0  15189   0   
5            rsv_b     cutadapt           ivar   NaN  15154.0  15189   0   
0            rsv_b  trimmomatic       bcftools   NaN  14968.0  15098  10   
1            rsv_b     cutadapt       bcftools   NaN  14968.0  15098  10   
2            rsv_b  trimmomatic      freebayes   NaN  14968.0  15098  10   
3            rsv_b     cutadapt      freebayes   NaN  14968.0  15098  10   
4   sars_cov_2_139  trimmomatic           ivar   NaN  29807.0  29821   0   
5   sars_cov_2_139     cutadapt           ivar   NaN  29807.0  29821   0   
2   sars_cov_2_139  trimmomatic      freebayes   NaN  29781.0  29821  26   
3   sars_cov_2_139     cutadapt      freebayes   NaN  29781.0  29821  26   
0   sars_cov_2_139  trimmomatic       bcftools   NaN  29769.0  29808  12   
1   sars_cov_2_139     cutadapt       bcftools   NaN  29769.0  29808  12   
4   sars_cov_2_122  trimmomatic           ivar   NaN  29831.0  29833   0   
5   sars_cov_2_122     cutadapt           ivar   NaN  29831.0  29833   0   
2   sars_cov_2_122  trimmomatic      freebayes   NaN  29805.0  29833  26   
3   sars_cov_2_122     cutadapt      freebayes   NaN  29805.0  29833  26   
0   sars_cov_2_122  trimmomatic       bcftools   NaN  29789.0  29818  12   
1   sars_cov_2_122     cutadapt       bcftools   NaN  29787.0  29817  12   
2  sars_cov_2_4964  trimmomatic      freebayes   NaN  29833.0  29840   0   
3  sars_cov_2_4964     cutadapt      freebayes   NaN  29833.0  29840   0   
4  sars_cov_2_4964  trimmomatic           ivar   NaN  29833.0  29840   0   
5  sars_cov_2_4964     cutadapt           ivar   NaN  29831.0  29839   0   
0  sars_cov_2_4964  trimmomatic       bcftools   NaN  29815.0  29831   0   
1  sars_cov_2_4964     cutadapt       bcftools   NaN  29815.0  29831   0   

  mismatch N_count  score_agg sample_id pairwiseAligner_set_score  \
2        4       4    13420.0      1337      match1_mis-1_o-1_e-1   
3        4       4    13420.0      1337      match1_mis-1_o-1_e-1   
4        4       4    13420.0      1337      match1_mis-1_o-1_e-1   
5        4       4    13420.0      1337      match1_mis-1_o-1_e-1   
0        5       4    13417.0      1337      match1_mis-1_o-1_e-1   
1        5       4    13417.0      1337      match1_mis-1_o-1_e-1   
2        0       1    13179.0      2129      match1_mis-1_o-1_e-1   
3        0       1    13179.0      2129      match1_mis-1_o-1_e-1   
0        1       1    13176.0      2129      match1_mis-1_o-1_e-1   
1        1       1    13176.0      2129      match1_mi

# merge: finalne vythodnotenia (Trimmomatic + variant callers oba s naj nastavenim) so skore s mbs nastavenim pre dany variant caller

In [8]:
# Trimmomatic


# najlepsie vyhodnitenie: trimmers + variant callers oba s naj nastavenim
path_trimm_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_2024_4_16_trimm_vc.tsv'
path_trimm_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_' + d + '_trimm_vc.tsv'
scores_df_all_viruses = pd.read_csv(path_trimm_vc, sep='\t')


# skore pre vyhondotenia variant callerov (vsetky testovane hodnoty parametrov vsetkych variant callerov)
path_to_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_2024_3_28_variant_callers.tsv'
path_to_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_' + d + '_variant_callers.tsv'
scores_df_all_viruses_variant_callers = pd.read_csv(path_to_vc, sep='\t')


# vyhondotenia: skore zarovani pre vzorku 4964 s mbs nastavenim trimmomaticu a mbs nastavenim variant callerov, ale analyzovany len maly set citani
path_4964_trimmomatic_mbs_vc_mbs = './data/result_metrics/results_all_viruses/results_4964_match1_mis-1_o-2_e-2_2024_4_16_trimmomatic_mbs_vc_mbs.tsv'
vzorka4964_trimmomatic_mbs_vc_mbs = pd.read_csv(path_4964_trimmomatic_mbs_vc_mbs, sep='\t')


# filter only mbs nastavenuie Trimmomaticu a mbs nastavenie variant callerov
scores_df_vc_mbs = scores_df_all_viruses_variant_callers[scores_df_all_viruses_variant_callers.param == 'baseline_params']
# add that baseline params are fromc Trimmomatic
scores_df_vc_mbs['trimmer'] = 'trimmomatic'

# vymaz vzroku 4964, lebo pri vyhodnocovani nastaveni hodnot parametrov variant callerov, bola analyzovana cela (vsetkych vela citani)
scores_df_vc_mbs = scores_df_vc_mbs[scores_df_vc_mbs.virus != 'sars_cov_2_4964']


# pridaj do scores_df_all_viruses_variant_callers_mbs vzorku 4964 s mbs nastavenim trimmomaticu a mbs nastavenim variant callerov, ale analyzovany len maly set citani
scores_df_all_viruses_variant_callers_mbs = pd.concat([scores_df_vc_mbs, vzorka4964_trimmomatic_mbs_vc_mbs])


merged_scores_df = pd.concat([scores_df_all_viruses, scores_df_all_viruses_variant_callers_mbs])

# filter only Trimmomatic
merged_scores_df = merged_scores_df[merged_scores_df.trimmer == 'trimmomatic']

path_trimmomatic_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_' + d + '_trimmomatic_vc_mbs.tsv'
merged_scores_df.to_csv(path_trimmomatic_vc, sep='\t', index=False)


/tmp/ipykernel_591599/2549312783.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scores_df_vc_mbs['trimmer'] = 'trimmomatic'


# merge tables with scores for cutadapt with mbs parameter set and variant callers with mbs param set

In [4]:

for _, row in set_scores_PairwiseAligner_df.iterrows():
    
    match_score = row['match_score']
    mismatch_score = row['mismatch_score']
    open_gap_score = row['open_gap_score']
    extend_gap_score = row['extend_gap_score']

    pairwiseAligner_set_scores_name = 'match' + str(match_score) + '_mis' + str(mismatch_score) + '_o' \
                                        + str(open_gap_score) + '_e' + str(extend_gap_score)
    
    scores_df_all_viruses = pd.DataFrame(columns=['virus', 'trimmer', 'variant_caller', 'param', 'score', 'match', 'gap', 'mismatch', 'N_count', 
                      'score_agg', 'sample_id', 'pairwiseAligner_set_score', 'consensus_baseline', 'consensus_baseline_virus'])
    
    for virus in samples_names.keys():

        score_path = './data/result_metrics/' + virus + '/' + virus + '_' + samples_names[virus].split('-')[-1] \
                        + pairwiseAligner_set_scores_name + '_' + d + '_cutadapt_mbs_vc_mbs.tsv'
        
        if os.path.exists(score_path):

            scores_df = pd.read_csv(score_path, sep='\t')
            scores_df['virus'] = virus
            scores_df['sample_id'] = samples_id[virus]
            scores_df['pairwiseAligner_set_score'] = pairwiseAligner_set_scores_name

            # pridaj oznacenie toho vyhodnotenia, ktore predstavuje MBS (porovnavane samo so sebou)
            scores_df['consensus_baseline'] = 'MBS'
            scores_df['consensus_baseline_virus'] = 'MBS' + '_' + virus
            scores_df['param'] = 'baseline_params'
            
            # sort by score
            scores_df = scores_df.sort_values(by=['virus', 'score'], ascending=[True, False])
            
            scores_df_all_viruses = pd.concat([scores_df_all_viruses, scores_df])
        
        else:
            print('Path: ', score_path, ' does not exists')

    path_to_save = './data/result_metrics/results_all_viruses/results_all_viruses_' + pairwiseAligner_set_scores_name + '_' + d + '_cutadapt_mbs_vc_mbs.tsv'
    scores_df_all_viruses.to_csv(path_to_save, sep='\t', index=False)

scores_df_all_viruses

virus   trimmer variant_caller            param    score  match  \
1             h1n1  cutadapt      freebayes  baseline_params  13424.0  13428   
0             h1n1  cutadapt       bcftools  baseline_params  13422.0  13427   
2             h1n1  cutadapt           ivar  baseline_params  13418.0  13425   
1             h3n2  cutadapt      freebayes  baseline_params  13179.0  13179   
0             h3n2  cutadapt       bcftools  baseline_params  13177.0  13178   
2             h3n2  cutadapt           ivar  baseline_params  13177.0  13178   
2            rsv_a  cutadapt           ivar  baseline_params  15221.0  15221   
0            rsv_a  cutadapt       bcftools  baseline_params  14793.0  15011   
1            rsv_a  cutadapt      freebayes  baseline_params  14793.0  15011   
2            rsv_b  cutadapt           ivar  baseline_params  15138.0  15181   
0            rsv_b  cutadapt       bcftools  baseline_params  14960.0  15096   
1            rsv_b  cutadapt      freebayes  baseline_params  14960.0  15096   
2   sars_cov_2_139  cutadapt           ivar  baseline_params  29805.0  29820   
0   sars_cov_2_139  cutadapt       bcftools  baseline_params  29757.0  29808   
1   sars_cov_2_139  cutadapt      freebayes  baseline_params  29755.0  29821   
2   sars_cov_2_122  cutadapt           ivar  baseline_params  29827.0  29831   
1   sars_cov_2_122  cutadapt      freebayes  baseline_params  29779.0  29833   
0   sars_cov_2_122  cutadapt       bcftools  baseline_params  29775.0  29817   
1  sars_cov_2_4964  cutadapt      freebayes  baseline_params  29833.0  29840   
2  sars_cov_2_4964  cutadapt           ivar  baseline_params  29831.0  29839   
0  sars_cov_2_4964  cutadapt       bcftools  baseline_params  29815.0  29831   

  gap mismatch N_count  score_agg sample_id pairwiseAligner_set_score  \
1   0        4       4    13420.0      1337      match1_mis-1_o-2_e-2   
0   0        5       4    13417.0      1337      match1_mis-1_o-2_e-2   
2   0        7       7    13411.0      1337      match1_mis-1_o-2_e-2   
1   0        0       1    13179.0      2129      match1_mis-1_o-2_e-2   
0   0        1       1    13176.0      2129      match1_mis-1_o-2_e-2   
2   0        1       1    13176.0      2129      match1_mis-1_o-2_e-2   
2   0        0       0    15221.0      1658      match1_mis-1_o-2_e-2   
0   4      210       0    14579.0      1658      match1_mis-1_o-2_e-2   
1   4      210       0    14579.0      1658      match1_mis-1_o-2_e-2   
2   0       43      46    15095.0       902      match1_mis-1_o-2_e-2   
0   6      124       0    14830.0       902      match1_mis-1_o-2_e-2   
1   6      124       0    14830.0       902      match1_mis-1_o-2_e-2   
2   0       15      24    29790.0       139      match1_mis-1_o-2_e-2   
0  12       27      32    29718.0       139      match1_mis-1_o-2_e-2   
1  26       14      48    29715.0       139      match1_mis-1_o-2_e-2   
2   0        4      26    29823.0       122      match1_mis-1_o-2_e-2   
1  26        2      48    29751.0       122      match1_mis-1_o-2_e-2   
0  12       18      32    29745.0       122      match1_mis-1_o-2_e-2   
1   0        7       7    29826.0      4964      match1_mis-1_o-2_e-2   
2   0        8       8    29823.0      4964      match1_mis-1_o-2_e-2   
0   0       16       7    29799.0      4964      match1_mis-1_o-2_e-2   

  consensus_baseline consensus_baseline_virus                  param_name  
1                MBS                 MBS_h1n1  cutadapt_mbs_freebayes_mbs  
0                MBS                 MBS_h1n1   cutadapt_mbs_bcftools_mbs  
2                MBS                 MBS_h1n1       cutadapt_mbs_ivar_mbs  
1                MBS                 MBS_h3n2  cutadapt_mbs_freebayes_mbs  
0                MBS                 MBS_h3n2   cutadapt_mbs_bcftools_mbs  
2                MBS                 MBS_h3n2       cutadapt_mbs_ivar_mbs  
2                MBS                MBS_rsv_a       cutadapt_mbs_ivar_mbs  
0                MBS                M

# merge: finalne vythodnotenia (Cutadapt + variant callers oba s naj nastavenim) so skore s mbs nastavenim pre dany variant caller a mbs nastavenim Cutadaptu

In [10]:
# cutadapt

# vyber nastaveni: trimmers + variant callers :: oba s naj nastavenim
path_trimm_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_2024_4_16_trimm_vc.tsv'
path_trimm_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_' + d + '_trimm_vc.tsv'
scores_df_all_viruses = pd.read_csv(path_trimm_vc, sep='\t')
scores_df_all_viruses

# mbs cutadapt, mbs VC
path_to_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_2024_4_15_cutadapt_mbs_vc_mbs.tsv'
scores_df_all_viruses_cutadapt = pd.read_csv(path_to_vc, sep='\t')

# filter only mbs vyhodnotenie
scores_df_all_viruses_cutadapt = scores_df_all_viruses_cutadapt[scores_df_all_viruses_cutadapt.param == 'baseline_params']
# add that baseline params are from cutadapt
scores_df_all_viruses_cutadapt['trimmer'] = 'cutadapt'

merged_scores_df = pd.concat([scores_df_all_viruses, scores_df_all_viruses_cutadapt])

# filter only Cutadapt
merged_scores_df = merged_scores_df[merged_scores_df.trimmer == 'cutadapt']

path_cutadapt_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_' + d + '_cutadapt_vc_mbs.tsv'
merged_scores_df.to_csv(path_cutadapt_vc, sep='\t', index=False)
merged_scores_df

virus   trimmer variant_caller            param    score  match  \
1              h1n1  cutadapt      freebayes              NaN  13424.0  13428   
3              h1n1  cutadapt           ivar              NaN  13424.0  13428   
5              h1n1  cutadapt       bcftools              NaN  13422.0  13427   
7              h3n2  cutadapt      freebayes              NaN  13179.0  13179   
9              h3n2  cutadapt       bcftools              NaN  13177.0  13178   
11             h3n2  cutadapt           ivar              NaN  13177.0  13178   
13            rsv_a  cutadapt           ivar              NaN  15221.0  15221   
15            rsv_a  cutadapt       bcftools              NaN  14793.0  15011   
17            rsv_a  cutadapt      freebayes              NaN  14793.0  15011   
19            rsv_b  cutadapt           ivar              NaN  15154.0  15189   
21            rsv_b  cutadapt       bcftools              NaN  14960.0  15096   
23            rsv_b  cutadapt      freebayes              NaN  14960.0  15096   
25   sars_cov_2_139  cutadapt           ivar              NaN  29807.0  29821   
27   sars_cov_2_139  cutadapt       bcftools              NaN  29785.0  29813   
29   sars_cov_2_139  cutadapt      freebayes              NaN  29755.0  29821   
31   sars_cov_2_122  cutadapt           ivar              NaN  29831.0  29833   
32   sars_cov_2_122  cutadapt       bcftools              NaN  29819.0  29830   
35   sars_cov_2_122  cutadapt      freebayes              NaN  29779.0  29833   
37  sars_cov_2_4964  cutadapt      freebayes              NaN  29833.0  29840   
39  sars_cov_2_4964  cutadapt           ivar              NaN  29831.0  29839   
41  sars_cov_2_4964  cutadapt       bcftools              NaN  29819.0  29833   
0              h1n1  cutadapt      freebayes  baseline_params  13424.0  13428   
1              h1n1  cutadapt       bcftools  baseline_params  13422.0  13427   
2              h1n1  cutadapt           ivar  baseline_params  13418.0  13425   
3              h3n2  cutadapt      freebayes  baseline_params  13179.0  13179   
4              h3n2  cutadapt       bcftools  baseline_params  13177.0  13178   
5              h3n2  cutadapt           ivar  baseline_params  13177.0  13178   
6             rsv_a  cutadapt           ivar  baseline_params  15221.0  15221   
7             rsv_a  cutadapt       bcftools  baseline_params  14793.0  15011   
8             rsv_a  cutadapt      freebayes  baseline_params  14793.0  15011   
9             rsv_b  cutadapt           ivar  baseline_params  15138.0  15181   
10            rsv_b  cutadapt       bcftools  baseline_params  14960.0  15096   
11            rsv_b  cutadapt      freebayes  baseline_params  14960.0  15096   
12   sars_cov_2_139  cutadapt           ivar  baseline_params  29805.0  29820   
13   sars_cov_2_139  cutadapt       bcftools  baseline_params  29757.0  29808   
14   sars_cov_2_139  cutadapt      freebayes  baseline_params  29755.0  29821   
15   sars_cov_2_122  cutadapt           ivar  baseline_params  29827.0  29831   
16   sars_cov_2_122  cutadapt      freebayes  baseline_params  29779.0  29833   
17   sars_cov_2_122  cutadapt       bcftools  baseline_params  29775.0  29817   
18  sars_cov_2_4964  cutadapt      freebayes  baseline_params  29833.0  29840   
19  sars_cov_2_4964  cutadapt           ivar  baseline_params  29831.0  29839   
20  sars_cov_2_4964  cutadapt       bcftools  baseline_params  29815.0  29831   

    gap  mismatch  N_count  score_agg  sample_id pairwiseAligner_set_score  \
1     0         4        4    13420.0       1337      match1_mis-1_o-2_e-2   
3     0         4        4    13420.0       1337      match1_mis-1_o-2_e-2   
5     0         5        4    13417.0       1337      match1_mis-1_o-2_e-2   
7     0         0        1    13179.0       2129      match1_mis-1_o-2_e-2   
9     0         1        1    13176.0       2129      match1_mis-1_o-2_e-2   
11    0         1        1    13176.0       2129      match1_m

# merge trimmomaric a cutadapt: pre porovnanie medzi sebou
- nastavenia oboch aj mbs nastavenia oboch
-- ale porovanie len naj nastaveni oboch nastrojov
 

In [11]:
path_trimmomatic_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_2024_4_16_trimmomatic_vc_mbs.tsv'
path_cutadapt_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_2024_4_16_cutadapt_vc_mbs.tsv'

path_trimmomatic_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_' + d + '_trimmomatic_vc_mbs.tsv'
path_cutadapt_vc = './data/result_metrics/results_all_viruses/results_all_viruses_match1_mis-1_o-2_e-2_' + d + '_cutadapt_vc_mbs.tsv'

trimmomatic_vc_df = pd.read_csv(path_trimmomatic_vc, sep='\t')
cutadapt_vc_df = pd.read_csv(path_cutadapt_vc, sep='\t')

print(trimmomatic_vc_df.columns)
cutadapt_vc_df.columns

merged_trimmomatic_cutadapt_vc_mbs = pd.concat([trimmomatic_vc_df, cutadapt_vc_df], ignore_index=True)

path_to_save = './data/result_metrics/results_all_viruses/finalne_nastavenia/results_all_viruses_match1_mis-1_o-2_e-2_' + d + '_trimmomatic_cutadapt_vc.tsv'
merged_trimmomatic_cutadapt_vc_mbs.to_csv(path_to_save, sep='\t', index=False)

Index(['virus', 'trimmer', 'variant_caller', 'param', 'score', 'match', 'gap',
       'mismatch', 'N_count', 'score_agg', 'sample_id',
       'pairwiseAligner_set_score', 'consensus_baseline',
       'consensus_baseline_virus', 'param_name', 'param_val'],
      dtype='object')


In [12]:
merged_trimmomatic_cutadapt_vc_mbs

virus      trimmer variant_caller            param    score  \
0              h1n1  trimmomatic      freebayes              NaN  13424.0   
1              h1n1  trimmomatic           ivar              NaN  13424.0   
2              h1n1  trimmomatic       bcftools              NaN  13422.0   
3              h3n2  trimmomatic      freebayes              NaN  13179.0   
4              h3n2  trimmomatic       bcftools              NaN  13177.0   
..              ...          ...            ...              ...      ...   
79   sars_cov_2_122     cutadapt      freebayes  baseline_params  29779.0   
80   sars_cov_2_122     cutadapt       bcftools  baseline_params  29775.0   
81  sars_cov_2_4964     cutadapt      freebayes  baseline_params  29833.0   
82  sars_cov_2_4964     cutadapt           ivar  baseline_params  29831.0   
83  sars_cov_2_4964     cutadapt       bcftools  baseline_params  29815.0   

    match  gap  mismatch  N_count  score_agg  sample_id  \
0   13428    0         4        4    13420.0       1337   
1   13428    0         4        4    13420.0       1337   
2   13427    0         5        4    13417.0       1337   
3   13179    0         0        1    13179.0       2129   
4   13178    0         1        1    13176.0       2129   
..    ...  ...       ...      ...        ...        ...   
79  29833   26         2       48    29751.0        122   
80  29817   12        18       32    29745.0        122   
81  29840    0         7        7    29826.0       4964   
82  29839    0         8        8    29823.0       4964   
83  29831    0        16        7    29799.0       4964   

   pairwiseAligner_set_score consensus_baseline consensus_baseline_virus  \
0       match1_mis-1_o-2_e-2                NaN                      NaN   
1       match1_mis-1_o-2_e-2                NaN                      NaN   
2       match1_mis-1_o-2_e-2                NaN                      NaN   
3       match1_mis-1_o-2_e-2                NaN                      NaN   
4       match1_mis-1_o-2_e-2                NaN                      NaN   
..                       ...                ...                      ...   
79      match1_mis-1_o-2_e-2                MBS       MBS_sars_cov_2_122   
80      match1_mis-1_o-2_e-2                MBS       MBS_sars_cov_2_122   
81      match1_mis-1_o-2_e-2                MBS      MBS_sars_cov_2_4964   
82      match1_mis-1_o-2_e-2                MBS      MBS_sars_cov_2_4964   
83      match1_mis-1_o-2_e-2                MBS      MBS_sars_cov_2_4964   

                    param_name param_val  
0        trimmomatic_freebayes       NaN  
1             trimmomatic_ivar       NaN  
2         trimmomatic_bcftools       NaN  
3        trimmomatic_freebayes       NaN  
4         trimmomatic_bcftools       NaN  
..                         ...       ...  
79  cutadapt_mbs_freebayes_mbs       NaN  
80   cutadapt_mbs_bcftools_mbs       NaN  
81  cutadapt_mbs_freebayes_mbs       NaN  
82       cutadapt_mbs_ivar_mbs       NaN  
83   cutadapt_mbs_bcftools_mbs       NaN  

[84 rows x 16 columns]